In [1]:
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd


In [2]:
def is_number(s):
    """ Returns True is string is a number. """
    return s.replace('.','',1).isdigit()

In [89]:
data_dict = {}
source = BeautifulSoup(open("./data/download/AAA.html"), "html.parser")

ticker_code = "AAA"
# Parsing date
days = [datetime.strptime(x.get_text().strip(), "%Y-%m-%d")  for x in source.select("li div.row-time.noline")[1:]]
data_dict["ticker_code"] = ["AAA" for x in range(len(days))]
data_dict["date"] = days


# Parsing prices
prices = [(float(x.get_text().strip()) if is_number(x.get_text().strip()) else x.get_text().strip()) for x in source.select("li div.row1")]
data_dict["open"] = prices[6::6]
data_dict["highest"] = prices[7::6]
data_dict["lowest"] = prices[8::6]
data_dict["close"] = prices[9::6]
data_dict["average"] = prices[10::6]
data_dict["adjusted"] = prices[11::6]

# Parsing volume
volumes = [((float(x.get_text().strip())) if is_number(x.get_text().strip()) else None) for x in source.select("li div.row3")[2:]]
data_dict["trading_volume"] = volumes[0::2]
data_dict["put_through_volume"] = volumes[1::2]

df = pd.DataFrame(data_dict)
df.head(10)
df.to_csv(f"{ticker_code}.csv",index = None)

### Parsing Ticker Code

In [13]:
data_dict = {}
source = BeautifulSoup(open("./data/download/1.html"), "html.parser")

In [14]:
[x.get_text().strip() for x in source.select("tbody tr td span")]

['A32',
 '32 JSC',
 'Công ty cổ phần 32',
 'Personal & Household Goods',
 'UPCOM',
 'AAA',
 'An Phat., JSC',
 'Công ty Cổ phần Nhựa và Môi trường xanh An Phát',
 'Industrial Goods & Services',
 'HOSTC',
 'AAM',
 'MEKONGFISH',
 'Công ty Cổ phần Thủy sản Mekong',
 'Food & Beverage',
 'HOSTC',
 'AAV',
 'VTSRE.,HC',
 'Công ty cổ phần Việt Tiên Sơn Địa ốc',
 'Retail',
 'HNX',
 'ABB',
 'ABBANK',
 'Ngân hàng Thương mại Cổ phần An Bình',
 'Banks',
 'OTC',
 'ABC',
 'VMG',
 'Công ty cổ phần Truyền thông VMG',
 'Telecommunications',
 'UPCOM',
 'ABI',
 'ABIC',
 'Công ty Cổ phần Bảo hiểm Ngân hàng Nông nghiệp (ABIC)',
 'Insurance',
 'UPCOM',
 'ABR',
 'VIET BRAND',
 'Công ty Cổ phần Đầu tư Nhãn hiệu Việt',
 'Utilities',
 'UPCOM',
 'ABT',
 'AQUATEX BENTRE',
 'Công ty Cổ phần Xuất nhập khẩu thủy sản Bến Tre',
 'Food & Beverage',
 'HOSTC',
 'AC4',
 'ACC - 244 JSC',
 'Công ty cổ phần ACC - 244',
 'Personal & Household Goods',
 'UPCOM']

In [22]:
df = pd.DataFrame(([1,2,3], ["asdasd asd",'5',6]))

In [23]:
df.head()

,0,1,2
0,1,2,3
1,asdasd asd,5,6


In [24]:
for _, row in df.iterrows():
    print(row.to_list())

[1, 2, 3]
['asdasd asd', '5', 6]


In [1]:
import psycopg2
import pandas as pd
from datetime import datetime, timedelta
from pytz import timezone

In [32]:
db_username = "admin"
db_psw = "B@chan1987"
use_virtual_screen = False
# conn_string = f"host=127.0.0.1 dbname=vietnam_stock user={db_username} password={db_psw}"
conn_string = f"host=127.0.0.1 dbname=vietnam_stock user={db_username} password={db_psw}"
time_zone = "Asia/Saigon"
date_format = "%d/%m/%Y"
    
last_updated = "01/01/2010"
with psycopg2.connect(conn_string) as conn:
    conn.set_session(autocommit=True)
    with conn.cursor() as cur:
        cur.execute(
            "SELECT MAX(date) FROM historical_price")
        result = cur.fetchall()[0][0]
        if result:
            last_updated = (result + timedelta(days=1)).strftime(date_format)


In [33]:
last_updated

'01/01/2010'

In [41]:
def last_update(ticker_code):
    """
    Get the last updated time of latest record
    """
    last_updated = "01/01/2010"
    with psycopg2.connect(conn_string) as conn:
        conn.set_session(autocommit=True)
        with conn.cursor() as cur:
            cur.execute(
                f"SELECT MAX(date) FROM historical_price WHERE ticker_code = '{ticker_code}'")
            result = cur.fetchall()[0][0]
            if result:
                last_updated = (result + timedelta(days=1)
                                ).strftime(date_format)
    return last_updated

In [42]:
last_update('AAA')

'31/08/2019'

In [7]:
x = []
for i in range(10):
    x.append(datetime.now() + timedelta(days=-i))

In [9]:
df=pd.DataFrame({"date": x})

In [10]:
df.head()

,date
0,2019-09-02 21:14:58.909168
1,2019-09-01 21:14:58.909177
2,2019-08-31 21:14:58.909180
3,2019-08-30 21:14:58.909183
4,2019-08-29 21:14:58.909185


In [14]:
df.sort_values(by="date", inplace=True)

In [15]:
df

,date
9,2019-08-24 21:14:58.909196
8,2019-08-25 21:14:58.909193
7,2019-08-26 21:14:58.909191
6,2019-08-27 21:14:58.909189
5,2019-08-28 21:14:58.909187
4,2019-08-29 21:14:58.909185
3,2019-08-30 21:14:58.909183
2,2019-08-31 21:14:58.909180
1,2019-09-01 21:14:58.909177
0,2019-09-02 21:14:58.909168


In [17]:
txt = "Page 1/11 12345 ... >>>"

In [18]:
last_page = txt.split(' ')[1].split('/')[1]

In [20]:
int(last_page)

11